In [20]:
queries = {
    "series_with_start_date": """
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX wd: <http://www.wikidata.org/entity/>
        SELECT ?series ?seriesLabel ?startDate WHERE {
          ?series wdt:P31 wd:Q5398426.
          OPTIONAL { ?series wdt:P577 ?startDate. }
          SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
          FILTER (BOUND(?startDate))
        } ORDER BY ?startDate
        LIMIT 10
    """,
    "genre": """
        SELECT ?series ?seriesLabel ?genreLabel WHERE {
          ?series wdt:P31 wd:Q5398426.
          ?series wdt:P136 ?genre.
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
        } LIMIT 10
    """,
    "actors": """
        SELECT ?series ?seriesLabel ?actorLabel WHERE {
          ?series wdt:P31 wd:Q5398426.
          ?series wdt:P161 ?actor.
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
        } LIMIT 10
    """,
    "directors": """
        SELECT ?series ?seriesLabel ?directorLabel WHERE {
          ?series wdt:P31 wd:Q5398426.
          ?series wdt:P57 ?director.
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
        } LIMIT 10
    """,
    "country_of_origin": """
        SELECT ?series ?seriesLabel ?countryLabel WHERE {
          ?series wdt:P31 wd:Q5398426.
          ?series wdt:P495 ?country.
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
        } LIMIT 10
    """,
    "original_language": """
        SELECT ?series ?seriesLabel ?languageLabel WHERE {
          ?series wdt:P31 wd:Q5398426.
          ?series wdt:P364 ?language.
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
        } LIMIT 10
    """,
    "number_of_seasons_episodes": """
        SELECT ?series ?seriesLabel ?numberOfSeasons ?numberOfEpisodes WHERE {
          ?series wdt:P31 wd:Q5398426.
          ?series wdt:P2437 ?numberOfSeasons.
          ?series wdt:P1113 ?numberOfEpisodes.
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
        } LIMIT 10
    """,
    "broadcasters": """
        SELECT ?series ?seriesLabel ?broadcasterLabel WHERE {
          ?series wdt:P31 wd:Q5398426.
          ?series wdt:P449 ?broadcaster.
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
        } LIMIT 10
    """,
    "imdb_rotten_tomatoes_ids": """
        SELECT ?series ?seriesLabel ?imdbId ?rottenTomatoesId WHERE {
          ?series wdt:P31 wd:Q5398426.
          OPTIONAL { ?series wdt:P345 ?imdbId. }
          OPTIONAL { ?series wdt:P1258 ?rottenTomatoesId. }
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
        } LIMIT 10
    """,
    "oscar": """
        SELECT ?film ?filmLabel ?award ?awardLabel WHERE {
          ?film wdt:P31 wd:Q11424; # Type: Film
                wdt:P166 ?award. # A reçu une récompense
          ?award wdt:P279* wd:Q19020. # Type de récompense: Oscar (y compris sous-catégories)
          SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
        } LIMIT 100
    """
        }

def query_wikidata(query_name):
    if query_name not in queries:
        return pd.DataFrame()  # Retourne un DataFrame vide si la requête n'existe pas
    
    sparql_query = queries[query_name]
    url = "https://query.wikidata.org/sparql"
    
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, params={'format': 'json', 'query': sparql_query})
    
    if response.status_code != 200:
        print("Erreur lors de la requête SPARQL", response.status_code)
        return pd.DataFrame()  # Retourne un DataFrame vide en cas d'erreur
    
    data = response.json()
    results = data['results']['bindings']
    rows = []
    
    for result in results:
        row = {key: result[key]['value'] for key in result.keys()}
        rows.append(row)
    
    df = pd.DataFrame(rows)
    
    return df

In [21]:
# Exemple d'utilisation
df = query_wikidata("original_language")
df.head()

,series,seriesLabel,languageLabel
0,http://www.wikidata.org/entity/Q494,Beakman's World,English
1,http://www.wikidata.org/entity/Q723,Rookie Blue,English
2,http://www.wikidata.org/entity/Q961,More Than Life at Stake,Polish
3,http://www.wikidata.org/entity/Q990,Czterej pancerni i pies,Polish
4,http://www.wikidata.org/entity/Q1077,Star Trek: The Original Series,English
